In [7]:
!pip install pillow
!pip install numpy
!pip install -U scikit-learn
!python -m pip install "tensorflow<2.11"


                                              0.0/455.9 MB ? eta -:--:--
                                              0.1/455.9 MB 2.6 MB/s eta 0:02:53
                                              0.2/455.9 MB 2.6 MB/s eta 0:02:53
                                              0.2/455.9 MB 2.6 MB/s eta 0:02:53
                                              0.3/455.9 MB 1.4 MB/s eta 0:05:19
                                              0.4/455.9 MB 1.6 MB/s eta 0:04:39
                                              0.6/455.9 MB 2.0 MB/s eta 0:03:44
                                              0.7/455.9 MB 2.2 MB/s eta 0:03:23
                                              0.9/455.9 MB 2.4 MB/s eta 0:03:08
                                              1.1/455.9 MB 2.5 MB/s eta 0:03:01
                                              1.3/455.9 MB 2.8 MB/s eta 0:02:44
                                              1.4/455.9 MB 2.8 MB/s eta 0:02:44
                                              1

# Libraries

In [13]:
from PIL import Image
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import cv2
import torch


torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 SUPER'

# conversion in dataset

In [11]:
path='../Dataset/indoors/'
folder=os.listdir(path)

df = pd.DataFrame(columns=['scene','scan','name','path_rgb','path_depth','path_mask'])

for scene in folder:
    scans=os.listdir(path+'/'+scene)
    for scan in scans:
        files=os.listdir(path+'/'+scene+'/'+scan)
        for file in files:
            if file[-4:]=='.png':
                #print(file[:-4])
                path_base=path+'/'+scene+'/'+scan+'/'+file[:-4]
                #print(str(path_base+'.png'), str(path_base+'_depth.npy',str(path_base+'_depth_mask.npy'))

                df.loc[len(df)]={'scene':str(scene),'scan':scan,'name':file[:-4]
                                  ,'path_rgb': str(path_base+'.png'),
                                  'path_depth': str(path_base+'_depth.npy'),
                                  'path_mask':str(path_base+'_depth_mask.npy')}

df.to_csv("../Csv/path_images.csv",index=False)
print(len(df))
df.head(2)

2155


,scene,scan,name,path_rgb,path_depth,path_mask
0,scene_00000,scan_00000,00000_00000_indoors_020_050,../Dataset/indoors//scene_00000/scan_00000/000...,../Dataset/indoors//scene_00000/scan_00000/000...,../Dataset/indoors//scene_00000/scan_00000/000...
1,scene_00000,scan_00000,00000_00000_indoors_030_030,../Dataset/indoors//scene_00000/scan_00000/000...,../Dataset/indoors//scene_00000/scan_00000/000...,../Dataset/indoors//scene_00000/scan_00000/000...


# Split dataset

In [12]:
df=pd.read_csv("../Csv/path_images.csv")
img_rgbs = []
img_rgbds = []
depth_maps=[]
for index,element in df[0:100].iterrows():
    img_rgb = Image.open(element['path_rgb'])
    img_rgbd = np.load(element['path_depth'])
    img_norm = np.load(element['path_mask'])
    img_rgbs.append(np.array(img_rgb))
    img_rgbds.append(img_rgbd)
    depth_maps.append(img_norm)

X_train_rgb, X_test_rgb, y_train_depth, y_test_depth, y_train_rgbds, y_test_rgbds = train_test_split(img_rgbs, depth_maps, img_rgbds, test_size=0.3, random_state=42)

In [13]:
X_train_rgb[0].shape , y_train_depth[0].shape, y_train_rgbds[0].shape # list of nump.array

((768, 1024, 3), (768, 1024), (768, 1024, 1))

In [11]:
tf.random.set_seed(123)
HEIGHT = 256
WIDTH = 256
LR = 0.0002
EPOCHS = 30
BATCH_SIZE = 32

In [18]:
df.head(1)
len(df) , df.columns ,

(2155,
 Index(['scene', 'scan', 'name', 'path_rgb', 'path_depth', 'path_mask'], dtype='object'))

In [49]:
class DataGenerator(tf.keras.utils.Sequence): # eredita dalla sequence
    
    def __init__(self, data, batch_size=6, dim=(768, 1024), n_channels=3, shuffle=True):
        self.data = data
        self.indices = self.data.index.tolist()
        self.dim = dim
        self.n_channels = n_channels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.min_depth = 0.1
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        if (index + 1) * self.batch_size > len(self.indices): # togliere gli elementi dell ultimo batch
            self.batch_size = len(self.indices) - index * self.batch_size # Generate one batch of data
        
        index = self.indices[index * self.batch_size : (index + 1) * self.batch_size]# Generate indices of the batch
        batch = [self.indices[k] for k in index] # Find list of IDs
        x, y = self.data_generation(batch)
        return x, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def load(self, image_path, depth_map, mask):
        image_ = cv2.imread(image_path)
        #image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
        image_ = cv2.resize(image_, self.dim)
        image_ = tf.image.convert_image_dtype(image_, tf.float32)

        depth_map = np.load(depth_map).squeeze() # remove dimension
        
        #mask = np.load(mask)
        #mask = mask > 0
        #max_depth = min(300, np.percentile(depth_map, 99))
        #depth_map = np.clip(depth_map, self.min_depth, max_depth)
        #depth_map = np.log(depth_map, where=mask)
        #depth_map = np.ma.masked_where(~mask, depth_map)
        #depth_map = np.clip(depth_map, 0.1, np.log(max_depth))
        depth_map = cv2.resize(depth_map, self.dim)
        depth_map = np.expand_dims(depth_map, axis=2)
        depth_map = tf.image.convert_image_dtype(depth_map, tf.float32)
        
        return image_, depth_map

    def data_generation(self, batch):
        x = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, 1))

        for i, batch_id in enumerate(batch):
            x[i,], y[i,] = self.load(
                self.data["path_rgb"][batch_id],
                self.data["path_depth"][batch_id],
                self.data["path_mask"][batch_id],
            )

        return x, y

In [35]:
class DownscaleBlock(layers.Layer):
    def __init__(self, filters, kernel_size=(3, 3), padding="same", strides=1, **kwargs):
        super().__init__(**kwargs)
        self.convA = layers.Conv2D(filters, kernel_size, strides, padding)
        #self.convB = layers.Conv2D(filters, kernel_size, strides, padding)
        self.reluA = layers.LeakyReLU(alpha=0.2)
        #self.reluB = layers.LeakyReLU(alpha=0.2)
        self.bn2a = tf.keras.layers.BatchNormalization()
        #self.bn2b = tf.keras.layers.BatchNormalization()
        self.pool = layers.MaxPool2D((2, 2), (2, 2))

    def call(self, input_tensor):
        d = self.convA(input_tensor)
        x = self.bn2a(d)
        x = self.reluA(x)

        #x = self.convB(x)
        #x = self.bn2b(x)
        #x = self.reluB(x)
        #x += d
        
        p = self.pool(x)
        return x, p


class UpscaleBlock(layers.Layer):
    def __init__(
        self, filters, kernel_size=(3, 3), padding="same", strides=1, **kwargs ):
        super().__init__(**kwargs)
        
        self.us = layers.UpSampling2D((2, 2))
        self.convA = layers.Conv2D(filters, kernel_size, strides, padding)
        #self.convB = layers.Conv2D(filters, kernel_size, strides, padding)
        self.reluA = layers.LeakyReLU(alpha=0.2)
        #self.reluB = layers.LeakyReLU(alpha=0.2)
        self.bn2a = tf.keras.layers.BatchNormalization()
        #self.bn2b = tf.keras.layers.BatchNormalization()
        #self.conc = layers.Concatenate()

    def call(self, x, skip):
        x = self.us(x)
        concat = self.conc([x, skip])
        x = self.convA(concat)
        x = self.bn2a(x)
        x = self.reluA(x)

        #x = self.convB(x)
        #x = self.bn2b(x)
        #x = self.reluB(x)

        return x


class BottleNeckBlock(layers.Layer):
    def __init__(
        self, filters, kernel_size=(3, 3), padding="same", strides=1, **kwargs
    ):
        super().__init__(**kwargs)
        self.convA = layers.Conv2D(filters, kernel_size, strides, padding)
        #self.convB = layers.Conv2D(filters, kernel_size, strides, padding)
        self.reluA = layers.LeakyReLU(alpha=0.2)
        #self.reluB = layers.LeakyReLU(alpha=0.2)

    def call(self, x):
        x = self.convA(x)
        x = self.reluA(x)
        #x = self.convB(x)
        #x = self.reluB(x)
        return x

In [45]:
class DepthEstimationModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.ssim_loss_weight = 0.85
        self.l1_loss_weight = 0.1
        self.edge_loss_weight = 0.9
        self.loss_metric = tf.keras.metrics.Mean(name="loss")
        f = [16, 32, 64, 128, 256]
        self.downscale_blocks = [
            DownscaleBlock(f[0]),
            DownscaleBlock(f[1]),
            DownscaleBlock(f[2]),
            DownscaleBlock(f[3]),
        ]
        self.bottle_neck_block = BottleNeckBlock(f[4])
        self.upscale_blocks = [
            UpscaleBlock(f[3]),
            UpscaleBlock(f[2]),
            UpscaleBlock(f[1]),
            UpscaleBlock(f[0]),
        ]
        self.conv_layer = layers.Conv2D(1, (1, 1), padding="same", activation="tanh")

    def calculate_loss(self, target, pred):
        # Edges
        dy_true, dx_true = tf.image.image_gradients(target)
        dy_pred, dx_pred = tf.image.image_gradients(pred)
        weights_x = tf.exp(tf.reduce_mean(tf.abs(dx_true)))
        weights_y = tf.exp(tf.reduce_mean(tf.abs(dy_true)))

        # Depth smoothness
        smoothness_x = dx_pred * weights_x
        smoothness_y = dy_pred * weights_y

        depth_smoothness_loss = tf.reduce_mean(abs(smoothness_x)) + tf.reduce_mean(abs(smoothness_y) )
        
        ssim_loss = tf.reduce_mean(1- tf.image.ssim(target, pred, max_val=WIDTH, filter_size=7, k1=0.01 ** 2, k2=0.03 ** 2))# Structural similarity (SSIM) index
        
        l1_loss = tf.reduce_mean(tf.abs(target - pred))# Point-wise depth

        loss = ((self.ssim_loss_weight * ssim_loss) + (self.l1_loss_weight * l1_loss)+ (self.edge_loss_weight * depth_smoothness_loss))

        return loss

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch_data):
        input, target = batch_data
        with tf.GradientTape() as tape:
            pred = self(input, training=True)
            loss = self.calculate_loss(target, pred)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result(),
        }

    def test_step(self, batch_data):
        input, target = batch_data

        pred = self(input, training=False)
        loss = self.calculate_loss(target, pred)

        self.loss_metric.update_state(loss)
        return {
            "loss": self.loss_metric.result(),
        }

    def call(self, x):
        c1, p1 = self.downscale_blocks[0](x)
        c2, p2 = self.downscale_blocks[1](p1)
        c3, p3 = self.downscale_blocks[2](p2)
        c4, p4 = self.downscale_blocks[3](p3)

        bn = self.bottle_neck_block(p4)

        u1 = self.upscale_blocks[0](bn, c4)
        u2 = self.upscale_blocks[1](u1, c3)
        u3 = self.upscale_blocks[2](u2, c2)
        u4 = self.upscale_blocks[3](u3, c1)

        return self.conv_layer(u4)

In [46]:
train_loader = DataGenerator(data=df[:260].reset_index(drop="true"), batch_size=(BATCH_SIZE), dim=(X_train_rgb[0].shape[0], X_train_rgb[0].shape[1]))

In [47]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LR, amsgrad=False,)
# Compile the model
model = DepthEstimationModel()
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")
model.compile(optimizer, loss=cross_entropy)

train_loader = DataGenerator(data=df[:260].reset_index(drop="true"), batch_size=BATCH_SIZE, dim=(HEIGHT, WIDTH))
validation_loader = DataGenerator(data=df[260:].reset_index(drop="true"), batch_size=BATCH_SIZE, dim=(HEIGHT, WIDTH))

In [52]:
model.fit(train_loader,epochs=EPOCHS,validation_data=validation_loader,)

ValueError: could not broadcast input array from shape (256,256) into shape (256,256,1)